# Building ANCOR Dataset

In [2]:
from french_crs.ancor2dataset import dataset_builder

dataset_params={
    # There are other three possible strategies: (1)"balanced", (2)"representative" and (3)"window"
    "strategy" : "balanced",
    "ancor_corpus_path" : "../DISTRIB_ANCOR/",
    # All possibilities are: ["corpus_OTG","corpus_UBS","corpus_ESLO" ,"corpus_ESLO_CO2"]
    "sub_corpus_filter" : ["corpus_OTG"], 
    # All possibilities are: ["DIRECTE", "INDIRECTE", "ANAPHORE"]
    "coreference_type_filter" : ["INDIRECTE"], 
    "dataset_output_folder" : "../datasets/",
    # This will be used only in window strategy
    "window_size" : 5 
}

dataset = dataset_builder(**dataset_params)
dataset.build_dataset(file_analysis_alert=True)
dataset.merge_dataset(delete_original_after_merge=True)

Feature extraction from 2AG0578 in corpus_OTG sucessfuly finished.
Feature extraction from 1AP0071 in corpus_OTG sucessfuly finished.
Feature extraction from 2AP0247 in corpus_OTG sucessfuly finished.
Feature extraction from 2AP0092 in corpus_OTG sucessfuly finished.
Feature extraction from 1AG0570 in corpus_OTG sucessfuly finished.
Feature extraction from 1AG0564 in corpus_OTG sucessfuly finished.
Feature extraction from 1AG0362 in corpus_OTG sucessfuly finished.
Feature extraction from 1PF0424 in corpus_OTG sucessfuly finished.
Feature extraction from 1PF0418 in corpus_OTG sucessfuly finished.
Feature extraction from 1PF0419 in corpus_OTG sucessfuly finished.
Feature extraction from 1AG0565 in corpus_OTG sucessfuly finished.
Feature extraction from 1SB0126 in corpus_OTG sucessfuly finished.
Feature extraction from 3SB0011 in corpus_OTG sucessfuly finished.
Feature extraction from 2AG0584 in corpus_OTG sucessfuly finished.
Feature extraction from 1AP0099 in corpus_OTG sucessfuly finis

Feature extraction from 2AG0495 in corpus_OTG sucessfuly finished.
Feature extraction from 1AP0605 in corpus_OTG sucessfuly finished.
Feature extraction from 1PF0641 in corpus_OTG sucessfuly finished.
Feature extraction from 1PF0457 in corpus_OTG sucessfuly finished.
Feature extraction from 2AP0195 in corpus_OTG sucessfuly finished.
Feature extraction from 1AP0228 in corpus_OTG sucessfuly finished.
Feature extraction from 2AP0205 in corpus_OTG sucessfuly finished.
Feature extraction from 1AP0033 in corpus_OTG sucessfuly finished.
Feature extraction from 1AP0219 in corpus_OTG sucessfuly finished.
Feature extraction from 3AP0272 in corpus_OTG sucessfuly finished.
Feature extraction from 2SB0025 in corpus_OTG sucessfuly finished.
Feature extraction from 1AP0146 in corpus_OTG sucessfuly finished.
Feature extraction from 1AP0152 in corpus_OTG sucessfuly finished.
Feature extraction from 1AP0608 in corpus_OTG sucessfuly finished.
Feature extraction from 1PF0467 in corpus_OTG sucessfuly finis

Feature extraction from 1AP0323 in corpus_OTG sucessfuly finished.
Feature extraction from 1SB0088 in corpus_OTG sucessfuly finished.
Feature extraction from 1PF0429 in corpus_OTG sucessfuly finished.
Feature extraction from 1AG0390 in corpus_OTG sucessfuly finished.
Feature extraction from 1AG0555 in corpus_OTG sucessfuly finished.
Feature extraction from 1AP0242 in corpus_OTG sucessfuly finished.
Feature extraction from 1AP0054 in corpus_OTG sucessfuly finished.
Feature extraction from 1AP0083 in corpus_OTG sucessfuly finished.
Feature extraction from 1SB0300 in corpus_OTG sucessfuly finished.

Dataset saved in ../datasets/balanced_INDIRECTE/balanced_INDIRECTE_corpus_OTG.xlsx 


Dataset saved in ../datasets/balanced_INDIRECTE/balanced_INDIRECTE_Merged.xlsx 


All files ['../datasets/balanced_INDIRECTE/balanced_INDIRECTE_Merged.xlsx', '../datasets/balanced_INDIRECTE/balanced_INDIRECTE_corpus_OTG.xlsx'] deleted


In [1]:
merge_lists = []
import pandas as pd
import glob

files_list = [file for file in glob.glob("/Users/mehdi.mirzapour/French-CRS/datasets/balanced_Full/*.xlsx")]

for file in files_list:
    merge_lists.append(pd.read_excel(file, index_col=0))

mrg_file = pd.concat(merge_lists, ignore_index=True)
mrg_file.to_excel('/Users/mehdi.mirzapour/French-CRS/datasets/balanced_Full/FULL.xlsx')



# Dataset Splitting

In [ ]:
from model_training import dataset_splitter

# ds1=dataset_splitter("./ds_90_10_new/corpus_ANCOR_N90_P10.xlsx",
#                     "./ds_90_10_new/corpus_ANCOR_N90_P10_Train.xlsx",
#                     "./ds_90_10_new/corpus_ANCOR_N90_P10_Test.xlsx")

ds2=dataset_splitter("./ds_window_15/corpus_ANCOR_Window_5.xlsx",
                    "./ds_window_15/corpus_ANCOR_Window_5_Train.xlsx",
                   "./ds_window_15/corpus_ANCOR_Window_5_Test.xlsx")

# dict_files=ds1.dataset_splitter_by_file(sub_corpus_num=4, lower_rate=0.20,upper_rate=0.22, files_num=84)
dict_files=ds2.dataset_splitter_by_dict(dict_files["sub_corpus_files_train"],dict_files["sub_corpus_files_test"])


# Model Training

In [ ]:
from model_training import model_trainer

model=model_trainer( "./ds_window_all/corpus_ANCOR_Window_ANAPHORE_Train.xlsx",
                     "./ds_window_5/corpus_ANCOR_Window_5_Test.xlsx",
                     "./ds_window_all/corpus_ANCOR_Window_Full_ANAPHORE_over_Test_Pred.xlsx",
                     "IS_CO_REF",
                     "IS_CO_REF"
                   )

model.columns_drop_list = ['m1_DEF', 'm2_DEF', 'ID_DEF',
                           'DISTANCE_MENTION','DISTANCE_WORD',
                           'DISTANCE_CHAR']

model.convert_columns_to_numeric()
model.train_model_random_forest(model_name="./Optimized_Models/Random_Forest_ANCOR_Window_ANAPHORE.model",max_depth=10, random_state=0,n_estimators=250)

# Model Testing

In [ ]:
from model_training import model_tester

model_parameter={
                "model_name" : "./Models/Random_Forest_(Normal)_OTG_Neg_90_Pos_10.model",
                "input_file" : "./Datasets/corpus_ALL_Window_30_Test.xlsx",
                "output_file" : "./Datasets/corpus_ALL_Window_30_Test_Called_Seperately.xlsx",
                "column_gold" : "IS_CO_REF",
                "column_outcome" : "Prediction",
                "threshold" : 0.5
                }

model=model_tester(**model_parameter)
model.apply_model_to_dataset()